<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
# Импорт библиотек для работы с sql
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Импорт данных для соединения с БД
from data import auth_data

DBNAME = auth_data.DBNAME_PROJECT
USER = auth_data.USER
PASSWORD = auth_data.PASSWORD
HOST = auth_data.HOST
PORT = auth_data.PORT

In [3]:
# Коннект к базе (предложенный в курсе вариант выдает ошибку в моем случае)
connection = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')

In [4]:
# Импорт библиотек для построения графиков
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

%matplotlib inline

In [5]:
# Имена таблиц
table_vacancies = 'vacancies'
table_employers = 'employers'
table_areas = 'areas'
table_industries = 'industries'
table_employers_industries = 'employers_industries'

In [6]:
from IPython.core.display import HTML
from IPython.display import display

def show_and_save_fig(fig, file_name):
    """Показывает и сохраняет график.

    Args:
        fig (Figure): объект с графиком
        file_name (string): имя файла графика без расширения
    """
    path_image = f'./plotly/{file_name}.png'
    path_html = f'./plotly/{file_name}.html'
    
    fig.show()
    fig.write_image(path_image)
    fig.write_html(path_html)
    
    display(HTML(f'<img src="{path_image}" /><br />'))
    display(HTML(f'<em>html графика в файле <a href="{path_html}" target="_blank">{path_html}</a></em>'))


# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [7]:
# текст запроса
query_3_1 = f'select count(*) from {table_vacancies}'

In [8]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, connection)
df_3_1

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [9]:
# текст запроса
query_3_2 = f'select count(*) from {table_employers}'

In [10]:
# результат запроса
df_3_2 = pd.read_sql_query(query_3_2, connection)
df_3_2

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [11]:
# текст запроса
query_3_3 = f'select count(*) from {table_areas}'

In [12]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, connection)
df_3_3

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [13]:
# текст запроса
query_3_4 = f'select count(*) from {table_industries}'

In [14]:
# результат запроса
df_3_4 = pd.read_sql_query(query_3_4, connection)
df_3_4

,count
0,294


***

## Выводы по предварительному анализу данных

Сравнение количества вакансий, работодателей, регионов и сфер деятельности между собой не даст, пожалуй, значимой информации.

Рассмотрим, как вакансии, регионы и сферы деятельности количественно распределяются между работодателями,

### Рассмотрим, как распределяются вакансии среди работодателей

Определим, сколько работодателей не имеют вакансий

In [15]:
query_employers_without_vacancies = f'''
select count(*) from
(
    select e.id
    from {table_employers} e
    left join {table_vacancies} v
        on e.id = v.employer_id
    group by e.id
    having count(v.id) = 0
) t1
'''
df_employers_without_vacancies = pd.read_sql_query(query_employers_without_vacancies, connection)
df_employers_without_vacancies

,count
0,8595


8 595 работодателей из 23 501 не имеют вакансий.

Определим, сколько вакансий у каждого работодателя (для тех, у кого они есть)

In [16]:
# Сколько у работодателя вакансий
query_vacancies_count_at_employers = f'''
select e.id e_id, e.name e_name, count(v.id) vacancies_cnt
from {table_employers} e
    join {table_vacancies} v
        on e.id = v.employer_id
group by e.id
order by vacancies_cnt desc
'''
df_vacancies_count_at_employers = pd.read_sql_query(query_vacancies_count_at_employers, connection)
df_vacancies_count_at_employers.head(10)

,e_id,e_name,vacancies_cnt
0,1740,Яндекс,1933
1,2748,Ростелеком,491
2,78638,Тинькофф,444
3,3529,СБЕР,428
4,39305,Газпром нефть,331
5,197135,ИК СИБИНТЕК,327
6,3776,МТС,292
7,69961,DataArt,247
8,5390761,Совкомбанк Технологии,204
9,3177,Первый Бит,176


Определим основные метрики распределения вакансий между работодателями

In [17]:
df_vacancies_count_at_employers['vacancies_cnt'].describe()

count    14906.000000
mean         3.300483
std         19.224989
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max       1933.000000
Name: vacancies_cnt, dtype: float64

По выводу таблицы и метрик видно, что максимальное количество вакансий у Яндекса - 1953.\
Следующим идет Ростелеком - 491, и между ними большой разрыв.\
Верхнее значение 1953 искажает статистику.\
Попробуем посмотреть метрики и построить график без него.

In [18]:
# Исключим из запроса данные Яндекса
query_vacancies_count_at_employers_2 = f'''
select e.id e_id, e.name e_name, count(v.id) vacancies_cnt
from {table_employers} e
    join {table_vacancies} v
        on e.id = v.employer_id
group by e.id
having count(v.id) < 1933
order by vacancies_cnt desc
'''
df_vacancies_count_at_employers_2 = pd.read_sql_query(query_vacancies_count_at_employers_2, connection)
df_vacancies_count_at_employers_2.head(10)

,e_id,e_name,vacancies_cnt
0,2748,Ростелеком,491
1,78638,Тинькофф,444
2,3529,СБЕР,428
3,39305,Газпром нефть,331
4,197135,ИК СИБИНТЕК,327
5,3776,МТС,292
6,69961,DataArt,247
7,5390761,Совкомбанк Технологии,204
8,3177,Первый Бит,176
9,139,IBS,175


Построим график с гистограммой и коробчатой диаграммой, который покажет, как распределяется количество вакансий между работодателями.

Чтобы не дублировать код в дальнейшем, напишем функцию, которая будет строить график.

In [19]:
def show_distribution(*, df, distribution_title, x_title, y_title, column_name, file_name):
    """Выводит график и метрики расределения какой-либо величины.
    Также сохраняет график как картинку и как html файл для просмотра на github.com

    Args:
        df (Data Frame): данные распределения
        distribution_title (string): Заголовок графика        
        x_title (string): Подпись по оси Х
        y_title (string): Подпись по оси Y
        column_name (string): Имя колонки, в которой содержаться данные о распределяемой величине
        file_name (string): Имя файла, в который будет сохранен график
    """  
    fig = make_subplots(rows=2, cols=1, subplot_titles=("Гистограмма", "Коробчатая диаграмма"))
    fig.update_xaxes(range=[df[column_name].min(), df[column_name].max()])
    
    fig.add_trace(go.Histogram(x=df[column_name]), row=1, col=1)
    fig.add_trace(go.Box(x=df[column_name], name=''), row=2, col=1)
    fig.update_layout(
        title=distribution_title,
        showlegend=False,
    )
    fig.update_xaxes(title=x_title, col=1, row=1)
    fig.update_yaxes(title=y_title, col=1, row=1)
    fig.update_xaxes(title=x_title, col=1, row=2)

    show_and_save_fig(fig, file_name)

    display(HTML('<strong>Основные метрики расределения<strong>'))
    display(df[column_name].describe())

Построим график.

In [20]:
show_distribution(
    df = df_vacancies_count_at_employers_2,    
    distribution_title = 'Распределение количества вакансий между работодателями',
    x_title = 'Количество вакансий на работодателя',
    y_title = 'Количество работодателей',
    column_name = 'vacancies_cnt',
    file_name = '03_vacancies_count_at_employers_2'
)

count    14905.000000
mean         3.171016
std         10.943485
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        491.000000
Name: vacancies_cnt, dtype: float64

Из графика (при наведении) видно, что большинство работодателей (8699) имеют одну вакансию.\
И после 127 вакансий на работодателя плотность распределения падает.

Распределение не является нормальным.\
50-й процентиль находится в значении 1,\
75-й в 2

### Рассмотрим, как распределяется между работодателями количество охваченных регионов

In [21]:
# В скольких регионах размещают вакансии работодатели
query_areas_count_at_employers = f'''
select e.id e_id, e.name e_name, count(distinct v.area_id) areas_cnt
from {table_employers} e
    join {table_vacancies} v
        on e.id = v.employer_id
group by e_id
order by areas_cnt desc
'''
df_areas_count_at_employers = pd.read_sql_query(query_areas_count_at_employers, connection)
df_areas_count_at_employers.head(10)

,e_id,e_name,areas_cnt
0,1740,Яндекс,181
1,2748,Ростелеком,152
2,5724811,Спецремонт,116
3,5130287,Поляков Денис Иванович,88
4,3682876,ООО ЕФИН,71
5,7944,Совкомбанк,63
6,3776,МТС,55
7,53797,"ЭФКО, Управляющая компания",49
8,3776815,КРОН,48
9,4352,Почта России,48


Построим график и выведем метрики расределения.

In [22]:
show_distribution(
    df = df_areas_count_at_employers,
    distribution_title = 'В скольких регионах размещает вакансии работодатель',    
    x_title = 'Количество регионов, охваченных работодателем',
    y_title = 'Количество работодателей',
    column_name = 'areas_cnt',
    file_name = '03_areas_count_at_employers'
)

count    14906.000000
mean         1.580169
std          3.312047
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max        181.000000
Name: areas_cnt, dtype: float64

Максимальное значение - 181 регион у Яндекса.\
При наведении на график видно, что 12 488 работодателей размещают вакансии только в одном регионе.\
После значения 29 регионов плотность распределения падает.

Распределение не является нормальным.\
50-й и 75-й процентили находятся в единице.

### Посмотрим, сколько сфер деятельности имеет Яндекс

Идентификатор Яндекса в таблице с работодателями **1740**. 

In [23]:
query_yandex_industries = f'''
select i.id, i.name
from {table_industries} i
    join {table_employers_industries} ei
        on i.id = ei.industry_id
where ei.employer_id = 1740
'''
df_yandex_industries = pd.read_sql_query(query_yandex_industries, connection)
df_yandex_industries

,id,name
0,7.541,"Интернет-компания (поисковики, платежные систе..."


Сферы деятельности самой крупной компании объеденены в одну строку.\
Чтобы получить более точные данные нужно изменять таблицу и разделять сферы деятельности на отдельные строки.\
Поэтому детальное рассмотрение количества вакансий у работодателей тут проводить не будем.

### Рассмотрим кореляцию между количеством вакансий и охваченных регионов у работодателей

Построим таблицу, в которой для каждого работодателя определим:
- сколько у него вакансий, 
- в каком количестве регионов он их размещает 

In [24]:
# соединим 2 таблицы в одну
query_vacancies_and_areas_count_at_employers = f'''
select t1.e_id, t1.e_name, t1.vacancies_cnt, t2.areas_cnt
from ({query_vacancies_count_at_employers}) t1
    join ({query_areas_count_at_employers}) t2
        on t1.e_id = t2.e_id
order by t1.vacancies_cnt desc, t2.areas_cnt desc
'''
df_vacancies_and_areas_count_at_employers = pd.read_sql_query(query_vacancies_and_areas_count_at_employers, connection)
df_vacancies_and_areas_count_at_employers.head(10)

,e_id,e_name,vacancies_cnt,areas_cnt
0,1740,Яндекс,1933,181
1,2748,Ростелеком,491,152
2,78638,Тинькофф,444,43
3,3529,СБЕР,428,24
4,39305,Газпром нефть,331,24
5,197135,ИК СИБИНТЕК,327,46
6,3776,МТС,292,55
7,69961,DataArt,247,2
8,5390761,Совкомбанк Технологии,204,26
9,3177,Первый Бит,176,43


Построим диаграмму рассеяния с полученными данными.

In [25]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_vacancies_and_areas_count_at_employers['vacancies_cnt'],
    y=df_vacancies_and_areas_count_at_employers['areas_cnt'],
    name='Точки с количеством вакансий и охваченных регионов у работодателей',
))

x = list(range(0, 101))
fig.add_trace(go.Scatter(x=x, y=x, name='Прямая х = у'))

fig.update_layout(
    title='Зависимость количества вакансий от количества охваченных регионов у работодателей',
    xaxis_title='Количество вакансий',
    yaxis_title='Количество регионов',
    legend_orientation='h',
    legend=dict(y=-.2)
)

show_and_save_fig(fig, '03_vacancies_and_areas_count_at_employers')

По графику видно, что есть зависимость между количеством регионов и количеством вакансий.\
Чем больше одна величина, тем больше и другая.\
Несколько работодателей (первые 8 из таблицы выше) сильно отрываются от других.\
Явное исключение составляет работодатель DataArt (247 вакансий и 2 региона).

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [26]:
# текст запроса
query_4_1 = f'''
    select a.id, a.name area, count(v.id) cnt
    from {table_areas} a
        left join {table_vacancies} v
            on a.id = v.area_id
    group by a.id
    order by cnt desc
'''

In [27]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1.head(10)

,id,area,cnt
0,1,Москва,5333
1,2,Санкт-Петербург,2851
2,1002,Минск,2112
3,4,Новосибирск,2006
4,160,Алматы,1892
5,3,Екатеринбург,1698
6,66,Нижний Новгород,1670
7,88,Казань,1415
8,53,Краснодар,1301
9,78,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [28]:
# текст запроса
query_4_2 = f'''
    select count(*) 
    from {table_vacancies}
    where (salary_from is not null) or (salary_to is not null)
'''

In [29]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, connection)
df_4_2

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [30]:
# текст запроса
query_4_3 = f'''
    select round(avg(salary_from)) avg_salary_from, round(avg(salary_to)) avg_salary_to
    from {table_vacancies}
'''

In [31]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [32]:
# текст запроса
query_4_4 = f'''
    select schedule, employment, count(id)
    from {table_vacancies}
    group by schedule, employment
    order by count desc
'''

In [33]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [34]:
# текст запроса
query_4_5 = f'''
    select experience, count(*) cnt
    from {table_vacancies}
    group by experience
    order by cnt
'''

In [35]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Выводы по детальному анализу вакансий

В предложенных выше запросах рассмотрены довольно разнообразные данные.\
Сделать по ним общий вывод сложно.\
Рассмотрим более детально некоторые направления.

### Рассмотрим подробнее распределение вакансий по регионам

Посмотрим, сколько регионов не имеют вакансий.

In [36]:
query_areas_without_vacancies = f'''
select count(*) from
(
    select a.id
    from {table_areas} a
        left join {table_vacancies} v
            on a.id = v.area_id
    group by a.id
    having count(v.id) = 0
) t1
'''

df_areas_without_vacancies = pd.read_sql_query(query_areas_without_vacancies, connection)
df_areas_without_vacancies

,count
0,593


593 региона среди 1362 не имеют вакансий.

In [37]:
print(f'{round(593 * 100 / 1362)}% регионов не имеют вакансий')

44% регионов не имеют вакансий


Рассмотрим распределение вакансий по регионам, где они есть.

In [38]:
query_vacancies_count_at_areas = f'''
    select a.id a_id, a.name a_name, count(v.id) vacancies_cnt
    from {table_areas} a
        join {table_vacancies} v
            on a.id = v.area_id
    group by a.id
    order by vacancies_cnt desc
'''
df_vacancies_count_at_areas = pd.read_sql_query(query_vacancies_count_at_areas, connection)
df_vacancies_count_at_areas.head(10)

,a_id,a_name,vacancies_cnt
0,1,Москва,5333
1,2,Санкт-Петербург,2851
2,1002,Минск,2112
3,4,Новосибирск,2006
4,160,Алматы,1892
5,3,Екатеринбург,1698
6,66,Нижний Новгород,1670
7,88,Казань,1415
8,53,Краснодар,1301
9,78,Самара,1144


Построим график и выведем метрики распределения.

In [39]:
show_distribution(
    df = df_vacancies_count_at_areas,
    distribution_title = 'Распределение вакансий между регионами',    
    x_title = 'Количество вакансий в регионе',
    y_title = 'Количество регионов',
    column_name = 'vacancies_cnt',
    file_name = '04_vacancies_count_at_areas'
)

count     769.000000
mean       63.975293
std       296.650833
min         1.000000
25%         1.000000
50%         4.000000
75%        15.000000
max      5333.000000
Name: vacancies_cnt, dtype: float64

Больше всего вакансий в Москве (5 333).\
Почти в 2 раза больше, чем в предыдущем городе - Санкт-Петербурге (2 851).\
При наведении на график видно, что 667 регионов имеют от 1 до 49 вакансий. Это значение для большинства регионов.\
После значения 851 вакансий на регион плотность распределения падает.\
Вообще вид распределения похож на те графики, которые получены выше (основная масса имеет малые значения, и всего несколько субъектов сильно лидируют по значениям)

### Исследуем зарплаты в регионах

Выведем для каждого региона среднюю зарплату и количество вакансий в нем.\
Оставим только регионы, в которых больше 10 вакансий.

In [40]:
query_salary_at_areas = f'''
select
    a.id a_id,
    a.name a_name,
    round(avg((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from))/2)) salary_avg,
    count(v.id) vacancies_cnt
from {table_areas} a
    join {table_vacancies} v
        on a.id = v.area_id
where (v.salary_from is not null) or (v.salary_to is not null)
group by a_id
having count(v.id) > 10
order by salary_avg desc
'''
df_salary_at_areas = pd.read_sql_query(query_salary_at_areas, connection)
df_salary_at_areas.head(10)

,a_id,a_name,salary_avg,vacancies_cnt
0,94,Турция,235435.0,19
1,236,Кипр,223736.0,39
2,13,Армения,219570.0,106
3,146,Сербия,208423.0,12
4,2758,Тбилиси,207156.0,80
5,2814,Батуми,199834.0,18
6,2734,Иннополис,175882.0,17
7,2492,Баку,155747.0,51
8,1,Москва,134140.0,1592
9,2092,Лобня,121136.0,11


Построим график и посмотрим метрики распределения.

In [41]:
show_distribution(
    df = df_salary_at_areas,
    distribution_title = 'Распределение средней зарплаты в регионах',    
    x_title = 'Средняя зарплата в регионе',
    y_title = 'Количество регионов',
    column_name = 'salary_avg',
    file_name = '04_salary_at_areas'
)

count       184.000000
mean      71497.380435
std       34623.552690
min       22776.000000
25%       52534.000000
50%       64603.000000
75%       79767.250000
max      235435.000000
Name: salary_avg, dtype: float64

Распределение похоже на нормальное.\
В Москве не самая большая средняя зарплата среди общих данных. Но если брать только регионы РФ, то самая большая.\
Большинство регионов имеют зарплату от 70 000 до 80 000 р.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [42]:
# текст запроса
query_5_1 = f'''
    select e.name e_name, count(*) vacancies_cnt
    from {table_vacancies} v
        join {table_employers} e
            on v.employer_id = e.id
    group by e.id
    order by vacancies_cnt desc
    limit 5
'''

In [43]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1

,e_name,vacancies_cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [44]:
# текст запроса
query_5_2 = f'''
    select t1.id, t1.name, t1.e_count, t2.v_count
    FROM
        (select a.id, a.name, count(e.id) e_count
        from {table_areas} a
        left join {table_employers} e
            on a.id = e.area    
        group by a.id) t1,
        
        (select a.id, a.name, count(v.id) v_count
        from {table_areas} a
        left join {table_vacancies} v
            on a.id = v.area_id
        group by a.id) t2
    where t1.id = t2.id and t2.v_count = 0
    order by t1.e_count desc
    limit 1
'''

In [45]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

,id,name,e_count,v_count
0,113,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [46]:
# текст запроса
query_5_3 = f'''
    select e.id e_id, e.name e_name, count(distinct v.area_id) areas_cnt
    from {table_employers} e
        left join {table_vacancies}  v
            on e.id = v.employer_id
    group by e_id
    order by areas_cnt desc    
'''

In [47]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3.head(10)

,e_id,e_name,areas_cnt
0,1740,Яндекс,181
1,2748,Ростелеком,152
2,5724811,Спецремонт,116
3,5130287,Поляков Денис Иванович,88
4,3682876,ООО ЕФИН,71
5,7944,Совкомбанк,63
6,3776,МТС,55
7,53797,"ЭФКО, Управляющая компания",49
8,3776815,КРОН,48
9,4352,Почта России,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [48]:
# текст запроса
query_5_4 = f'''
    select count(*) from
    (
        select e.id e_id, e.name e_name, count(i.industry_id) industryes_cnt
        from {table_employers} e
            left join {table_employers_industries} i
                on e.id = i.employer_id
        group by e_id
        having count(i.industry_id) = 0
    ) t1
'''

In [49]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [50]:
# текст запроса
query_5_5 = f'''
    select e.id e_id, e.name e_name, count(i.industry_id) industryes_cnt
    from {table_employers} e
        join {table_employers_industries} i
            on e.id = i.employer_id
    group by e_id
    having count(i.industry_id) = 4
    order by e_name
    offset 2
    limit 1
'''

In [51]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

,e_id,e_name,industryes_cnt
0,64174,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [52]:
# текст запроса
query_5_6 = f'''
    select count(*) from
    (
        select e.id
        from {table_employers} e
            join {table_employers_industries} i
                on e.id = i.employer_id
        where i.industry_id = (select id from {table_industries} where name = 'Разработка программного обеспечения')
    ) t1
'''

In [53]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [54]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')

links = page.find('table', class_='standard sortable').find('tbody').select('td > a')
cities = [link.text for link in links]

print(cities)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград']


In [55]:
# текст запроса
query_5_7 = f'''
with t1 as
(
    select a.name, count(v.id) cnt
    from {table_employers} e
        join {table_vacancies} v
            on e.id = v.employer_id
                join {table_areas} a
                    on a.id = v.area_id
    where
        e.id = (select id from {table_employers} where name = 'Яндекс')
        and v.area_id in (select id from {table_areas} where name in ('{"','".join(cities)}'))
    group by a.id
    order by count(v.id)
)

select * from t1
union all
select 'Total', sum(cnt) from t1
'''

In [56]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Ростов-на-Дону,25.0
5,Казань,25.0
6,Пермь,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

## Выводы по анализу работодателей

Посмотрим подробнее на список регионов, в которых есть работодатели, но нет вакансий.

In [57]:
query_5c_1 = f'''
    select t1.id, t1.name, t1.e_count, t2.v_count
    FROM
        (select a.id, a.name, count(e.id) e_count
        from {table_areas} a
        left join {table_employers} e
            on a.id = e.area    
        group by a.id) t1,
        
        (select a.id, a.name, count(v.id) v_count
        from {table_areas} a
        left join {table_vacancies} v
            on a.id = v.area_id
        group by a.id) t2
    where t1.id = t2.id and t2.v_count = 0
    order by t1.e_count desc
'''
df_5c_1 = pd.read_sql_query(query_5c_1, connection)
df_5c_1.head(30)

,id,name,e_count,v_count
0,113,Россия,410,0
1,40,Казахстан,207,0
2,2019,Московская область,75,0
3,1438,Краснодарский край,19,0
4,1530,Ростовская область,18,0
5,16,Беларусь,18,0
6,9,Азербайджан,17,0
7,1624,Республика Татарстан,16,0
8,1679,Нижегородская область,16,0
9,97,Узбекистан,15,0


У некоторых работодателей указан регион уровня страны или области. \
В вакансиях же обычно указывают более точные регионы, те, где люди имеют прописку. \
То есть, есть такая особенность в заполнении данных. \
Для более точного анализа нужно указать локальные регионы тем работодателям, для которых это возможно.\
Или составить таблицу, в которой будет указано, какой регион к какой стране или области относится.

Данные по сферам деятельности не полные и требуют доработки.\
У 8419 работодателей из 23501 не указана сфера деятельности. Хотя она у них явно есть и скорее всего не одна.\
У Яндекса указана всего 1 сфера деятельности (несколько объеденино в одну строку)\
Для анализа статистики такие данные не подходят, нужно их дополнять для тех работодателей, для которых это возможно\
и разъединять в отдельные строки сферы деятельности, которые объеденены в одну запись. \
Но это не тема данной работы.

Среди 1933 вакансий Яндекса 485 находятся в городах милионниках.\
Всего же Яндекс размещает свои вакансии в 181 регионе.\
Для наглядности, как распределяются вакансии Яндекса по регионам, построим иерархическую диаграмму.

In [58]:
query_yandex_vacancies_in_regions = f'''
    select a.name, count(v.id) cnt
    from {table_employers} e
        join {table_vacancies} v
            on e.id = v.employer_id
                join {table_areas} a
                    on a.id = v.area_id
    where
        e.id = (select id from {table_employers} where name = 'Яндекс')        
    group by a.id
    order by count(v.id) desc
'''
df_yandex_vacancies_in_regions = pd.read_sql_query(query_yandex_vacancies_in_regions, connection)
fig = px.treemap(
    data_frame=df_yandex_vacancies_in_regions, #DataFrame
    path=['name'], #категориальный признак, для которого строится график
    values='cnt', #параметр, который сравнивается
    height=500, #высота
    width=1000, #ширина
    title='Распределение вакансий Яндекса по регионам' #заголовок
)

#отображаем график
show_and_save_fig(fig, '05_1_yandex_vacancies_in_regions')

А также построим гистограмму и коробчатую диаграмму.

In [59]:
show_distribution(
    df = df_yandex_vacancies_in_regions,
    distribution_title = 'Распределение вакансий Яндекса по регионам',    
    x_title = 'Количество вакансий в регионе',
    y_title = 'Количество регионов',
    column_name = 'cnt',
    file_name = '05_2_yandex_vacancies_in_region'
)

count    181.000000
mean      10.679558
std        8.978807
min        1.000000
25%        4.000000
50%        7.000000
75%       15.000000
max       54.000000
Name: cnt, dtype: float64

По графику видно, что в большинстве регионов (54 штуки) размещается по 5-9 вакансий.\
Распределение похоже на нормальное, с левосторонней ассиметрией.
Чего-то сильно необычного в этом распределении не наблюдается.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [60]:
# текст запроса
query_6_1 = f'''
    select count(*)
    from {table_vacancies}
    where name ilike '%%data%%' or name ilike '%%данн%%'
'''

In [61]:
# результат запроса
df_6_1 = pd.read_sql_query(query_6_1, connection)
df_6_1

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [62]:
# условие для фильтра вакансий по полю name (используем его для следующих запросов)
def get_filter_for_data_science(field_name):
    """Возвращает условие для фильтра вакансий с направлением Data Science

    Args:
        field_name (string): Имя поля, в котором находится название вакансии

    Returns:
        string: строка для вставки в where запроса
    """
    filter = f'''
        {field_name} ilike '%%data scientist%%' or
        {field_name} ilike '%%data science%%' or
        {field_name} ilike '%%исследователь данных%%' or
        ({field_name} like '%%ML%%' and {field_name} not ilike '%%html%%') or
        {field_name} ilike '%%machine learning%%' or
        {field_name} ilike '%%машинн%%обучен%%'
    '''
    return filter
    
filter_for_data_science = get_filter_for_data_science('name')

# текст запроса
query_6_2 = f'''
    select count(*)
    from {table_vacancies}
    where
        ({filter_for_data_science})
        and
        (
            name ilike '%%junior%%' or
            experience = 'Нет опыта' or
            employment = 'Стажировка'
        )    
'''

In [63]:
# результат запроса
df_6_2 = pd.read_sql_query(query_6_2, connection)
df_6_2

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [64]:
# текст запроса
query_6_3 = f'''
    select count(*)
    from {table_vacancies}
    where
        ({filter_for_data_science})
        and
        (
            key_skills like '%%SQL%%' or
            key_skills ilike '%%postgres%%'
        )    
'''

In [65]:
# результат запроса
df_6_3 = pd.read_sql_query(query_6_3, connection)
df_6_3

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [66]:
# текст запроса
query_6_4 = f'''
    select count(*)
    from {table_vacancies}
    where
        ({filter_for_data_science})
        and
        (
            key_skills ilike '%%Python%%'
        )    
'''

In [67]:
# результат запроса
df_6_4 = pd.read_sql_query(query_6_4, connection)
df_6_4

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [68]:
# текст запроса
query_6_5 = f'''
    select round(avg(array_length(string_to_array(key_skills, '\t'), 1)), 2)
    from {table_vacancies}
    where
        ({filter_for_data_science})
        and
        (
            key_skills is not null
        )
'''

In [69]:
# результат запроса
df_6_5 = pd.read_sql_query(query_6_5, connection)
df_6_5

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [70]:
# текст запроса
query_6_6 = f'''
    select experience, round(avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from))/2)) salary_avg
    from {table_vacancies}
    where
        ({filter_for_data_science})
        and
        (
            (salary_from is not null) or
            (salary_to is not null) 
        )
    group by experience
'''

In [71]:
# результат запроса
df_6_6 = pd.read_sql_query(query_6_6, connection)
df_6_6

,experience,salary_avg
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

## Выводы по предметному анализу

### Посмотрим, сколько всего вакансий имеет отношение к Data Science.

In [72]:
query_ds_vacancies_count = f'select count(*) from {table_vacancies} where ({filter_for_data_science})'
df_ds_vacancies_count = pd.read_sql_query(query_ds_vacancies_count, connection)
df_ds_vacancies_count

,count
0,480


In [73]:
print(f'Вакансии, имеющие отношение к Data Science, составляют {round(480 * 100 / 49197, 2)}% от общего количества вакансий.')

Вакансии, имеющие отношение к Data Science, составляют 0.98% от общего количества вакансий.


То есть, менее 1-го процента.

### Построим столбчатую диаграмму с данными о требуемых ключевых навыках

Воспользуемся данными, найденными выше.

In [74]:
fig = go.Figure()

fig.add_trace(go.Bar(x = ['Общее количество вакансий', 'Требуется SQL или postgres', 'Требуется Python'], y = [480, 201, 351]))

fig.update_layout(
    title='Соотношение вакансий с распространенными ключевыми навыками и их общего количества',
    xaxis_title='Требуемые навыки',
    yaxis_title='Количество вакансий',
)

show_and_save_fig(fig, '06_bar_of_key_skills')

Из рассмотренных навыков наиболее востребован Python.

### Посмотрим, для какого количества вакансий не указаны оба поля salary_from и salary_to

In [75]:
query_count_vacancies_without_salary = f'''
    select count(*) 
    from {table_vacancies}
    where
        ({filter_for_data_science})
        and
        (salary_from is null and salary_to is null)
'''
df_count_vacancies_without_salary = pd.read_sql_query(query_count_vacancies_without_salary, connection)
df_count_vacancies_without_salary

,count
0,418


In [76]:
print(f'Оба поля с зарплатой не указаны у {round(418*100/480, 2)}% вакансий, имеющих отношение к Data Science.')

Оба поля с зарплатой не указаны у 87.08% вакансий, имеющих отношение к Data Science.


### Посмотрим на соотношение количества вакансий и требуемого опыта

In [77]:
query_ds_experience = f'''
    select experience, count(*)
    from {table_vacancies}
    where
        ({filter_for_data_science})
    group by experience
'''
df_ds_experience = pd.read_sql_query(query_ds_experience, connection)
df_ds_experience

,experience,count
0,Более 6 лет,34
1,Нет опыта,28
2,От 1 года до 3 лет,223
3,От 3 до 6 лет,195


Построим столбчатую диаграмму для наглядности.

In [78]:
fig = go.Figure()

fig.add_trace(go.Bar(x = df_ds_experience['experience'], y = df_ds_experience['count']))

fig.update_layout(
    title='Количество вакансий для каждого требуемого опыта',
    xaxis_title='Требуемый опыт',
    yaxis_title='Количество вакансий',
)

show_and_save_fig(fig, '06_data_science_experience')

Вариант опыта от года до трех лет самый распространенный.\
Видимо, для работодателей это лучший вариант соотношения цена/качество.

# Общий вывод по проекту

## Проведем некоторые дополнительные исследования

### Посмотрим, какие работодатели больше набирают сотрудников направления Data Science

Составим таблицу, в которой соберем вместе 
- название работодателя, 
- его общее количество вакансий 
- количество вакансий направления Data Science
- процент вакансий  Data Science от общего количества вакансий работодателя

Попробуем таким образом выделить работодателей, которые больше нацелены на поиск Data Science специалистов.

In [79]:
# Таблица с количеством вакансий Data Science для каждого работодателя
filter_for_data_science_2 = get_filter_for_data_science('v.name')
query_ds_vacancies_cnt_at_employers = f'''
    select e.id e_id, e.name e_name, count(*) ds_vacancies_cnt
    from {table_employers} e
        join {table_vacancies} v
            on e.id = v.employer_id
    where ({filter_for_data_science_2})
    group by e_id
    order by ds_vacancies_cnt desc
'''
df_ds_vacancies_cnt_at_employers = pd.read_sql_query(query_ds_vacancies_cnt_at_employers, connection)
df_ds_vacancies_cnt_at_employers.head(10)

,e_id,e_name,ds_vacancies_cnt
0,3529,СБЕР,37
1,6189,Bell Integrator,25
2,4181,Банк ВТБ (ПАО),18
3,15478,VK,15
4,26624,Positive Technologies,11
5,1740,Яндекс,9
6,5966150,EvenBet Gaming,9
7,3127,МегаФон,8
8,851604,Andersen,7
9,2180,Ozon,6


In [80]:
# Таблица с общим количеством вакансий для каждого работодателя
query_all_vacancies_cnt_at_employers = f'''
    select e.id e_id, e.name e_name, count(*) all_vacancies_cnt
    from {table_employers} e
        join {table_vacancies} v
            on e.id = v.employer_id
    group by e_id
    order by all_vacancies_cnt desc
'''
df_all_vacancies_cnt_at_employers = pd.read_sql_query(query_all_vacancies_cnt_at_employers, connection)
df_all_vacancies_cnt_at_employers.head(5)

,e_id,e_name,all_vacancies_cnt
0,1740,Яндекс,1933
1,2748,Ростелеком,491
2,78638,Тинькофф,444
3,3529,СБЕР,428
4,39305,Газпром нефть,331


In [81]:
# Сводная таблица из первых двух
query_ds_procent_at_employers = f'''
    select
        t1.e_id,
        t1.e_name,
        t1.ds_vacancies_cnt,
        t2.all_vacancies_cnt,
        round(t1.ds_vacancies_cnt * 100 / t2.all_vacancies_cnt, 2) ds_procent
    from
        ({query_ds_vacancies_cnt_at_employers}) t1
            join ({query_all_vacancies_cnt_at_employers}) t2
                on t1.e_id = t2.e_id
    order by ds_vacancies_cnt desc, ds_procent desc
'''
df_ds_procent_at_employers = pd.read_sql_query(query_ds_procent_at_employers, connection)
df_ds_procent_at_employers.head(10)

,e_id,e_name,ds_vacancies_cnt,all_vacancies_cnt,ds_procent
0,3529,СБЕР,37,428,8.0
1,6189,Bell Integrator,25,121,20.0
2,4181,Банк ВТБ (ПАО),18,94,19.0
3,15478,VK,15,150,10.0
4,26624,Positive Technologies,11,73,15.0
5,5966150,EvenBet Gaming,9,25,36.0
6,1740,Яндекс,9,1933,0.0
7,3127,МегаФон,8,104,7.0
8,851604,Andersen,7,155,4.0
9,2575,Бэнкс Софт Системс,6,24,25.0


Наиболее нацелен на поиск Data Science специалистов СБЕР.\
Но надо заметить, что на значение процента в столбце  **ds_procent** нельзя ориентироваться как на самостоятельное значение.\
Потому что для Яндекса он показывает 0 из-за того, что общее количество вакансий Яндекса велико.\
Но тем не менее Яндекс точно заинтересован в таких специалистах, потому что он делает поиск, Алису и много других сервисов.

### Посмотрим, в каких городах больше набирают сотрудников направления Data Science

In [82]:
query_ds_vacancies_cnt_at_areas = f'''
    select a.id a_id, a.name a_name, count(*) ds_vacancies_cnt
    from {table_areas} a
        join {table_vacancies} v
            on a.id = v.area_id
    where ({filter_for_data_science_2})
    group by a.id
    order by ds_vacancies_cnt desc
'''
df_ds_vacancies_cnt_at_areas = pd.read_sql_query(query_ds_vacancies_cnt_at_areas, connection)
df_ds_vacancies_cnt_at_areas.head(5)

,a_id,a_name,ds_vacancies_cnt
0,1,Москва,217
1,2,Санкт-Петербург,64
2,4,Новосибирск,23
3,66,Нижний Новгород,20
4,88,Казань,16


В Москве и Санкт Петербурге, что ожидаемо.

## Общие выводы 
Что касается распределения вакансий и регионов по работодателям, то это распределение не является нормальным.\
Небольшое количество работодателей явно вырывается вперед.\
У большинства значения небольшие.\
Это же касается и распределения вакансий по регионам.\
Есть кореляция между количеством охваченных регионов и количеством вакансий у работодателей.

При распределении зарплат по регионам такой особенности не наблюдается.\
Здесь распределение больше похоже на нормальное, имеет левостороннюю ассиметрию.

## Мысли про дальнейшие исследования
В таблице со сферами деятельности **industries** некоторые данные объеденены в одну строку (например, у Яндекса).\
Чтобы исследовать данные, нужно разделить их на отдельные строки.\
Тогда можно было бы посмотреть, работодатели с какими сферами деятельности чаще ищут Data Science специалистов.\
И как количество сфер деятельности корелирует с количеством вакансий и охваченных регионов.

Для вакансий, которые имеют отношение к Data Science, можно было бы сделать отдельную таблицу, \
в которую вынести значения из key_skills в отдельные строки.\
Тогда можно было бы посмотреть, какие навыки наиболее востребованы.\
И подбирать вакансии, которые наиболее подходят к навыками у соискателя.

In [83]:
# закрываем соединение
connection.dispose()